## 2025/03/08

In [ ]:
print(df.columns)  # Prints the column names
print(df.head())  # Shows the first few rows

Index(['id', 'text', 'HS', 'TR', 'AG'], dtype='object')
      id                                               text  HS  TR  AG
0  18201  I swear I’m getting to places just in the nick...   0   0   0
1  18202  I’m an immigrant — and Trump is right on immig...   0   0   0
2  18203  #IllegalImmigrants #IllegalAliens #ElectoralSy...   1   0   1
3  18204  @DRUDGE_REPORT We have our own invasion issues...   1   0   1
4  18205  Worker Charged With Sexually Molesting Eight C...   0   0   0


In [ ]:
# Install required libraries (if not already available)
!pip install pandas numpy

import pandas as pd
import numpy as np
from google.colab import files

# Upload the dataset manually to Colab
uploaded = files.upload()  # Upload 'train_en.tsv'

# Load the dataset (TSV format with tab delimiter)
dataset_path = list(uploaded.keys())[0]  # Gets the uploaded file name
df = pd.read_csv(dataset_path, sep='\t', on_bad_lines='warn')

# Basic statistics (using 'HS' as the hate speech label)
total_samples = len(df)
class_distribution = df['HS'].value_counts(normalize=True) * 100  # Percentage
avg_text_length = df['text'].apply(lambda x: len(str(x).split())).mean()
unique_words = len(set(" ".join(df['text']).split()))

# Print statistics
print(f"Total number of samples: {total_samples}")
print(f"Class distribution (% of hate speech, HS):\n{class_distribution}")
print(f"Average number of words per text: {avg_text_length:.2f}")
print(f"Number of unique words: {unique_words}")

# Save statistics to a file and download it
with open("dataset_statistics.txt", "w") as f:
    f.write(f"Total number of samples: {total_samples}\n")
    f.write(f"Class distribution (% of hate speech, HS):\n{class_distribution.to_string()}\n")
    f.write(f"Average number of words per text: {avg_text_length:.2f}\n")
    f.write(f"Number of unique words: {unique_words}\n")

files.download("dataset_statistics.txt")  # Downloads the file

KeyboardInterrupt: 

In [ ]:
# Install required libraries
!pip install scikit-learn transformers datasets torch pandas numpy

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset
from google.colab import files

# Upload the dataset manually to Colab
uploaded = files.upload()  # Upload 'train_en.tsv'

# Load the dataset (TSV format with tab delimiter)
dataset_path = list(uploaded.keys())[0]  # Gets the uploaded file name
df = pd.read_csv(dataset_path, sep='\t', on_bad_lines='warn')

# Basic preprocessing function
def preprocess_text(text):
    text = str(text).lower()
    text = ''.join(c for c in text if c.isalnum() or c.isspace())
    return text

df['text'] = df['text'].apply(preprocess_text)

# Split into train and test sets (using 'HS' as the label)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# --- Baseline Model: TF-IDF + Logistic Regression ---
print("Training Baseline Model (TF-IDF + Logistic Regression)...")
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train = vectorizer.fit_transform(train_df['text'])
X_test = vectorizer.transform(test_df['text'])
y_train = train_df['HS']  # Correct: Training labels
y_test = test_df['HS']   # Fixed: Test labels (was train_df['HS'])

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)

print("Baseline Model Evaluation:")
print(classification_report(y_test, y_pred))

# --- Transformer Model: BERT ---
print("Training BERT Model...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

train_dataset = Dataset.from_pandas(train_df[['text', 'HS']].rename(columns={'HS': 'label'}))
test_dataset = Dataset.from_pandas(test_df[['text', 'HS']].rename(columns={'HS': 'label'}))

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"  # Disable W&B
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()
eval_results = trainer.evaluate()
print("BERT Model Evaluation:")
print(eval_results)

model.save_pretrained("bert_hate_speech_model")
tokenizer.save_pretrained("bert_hate_speech_model")
!zip -r bert_hate_speech_model.zip bert_hate_speech_model
files.download("bert_hate_speech_model.zip")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Saving train_en.tsv to train_en (1).tsv
Training Baseline Model (TF-IDF + Logistic Regression)...
Baseline Model Evaluation:
              precision    recall  f1-score   support

           0       0.78      0.87      0.82      1060
           1       0.78      0.65      0.71       740

    accuracy                           0.78      1800
   macro avg       0.78      0.76      0.76      1800
weighted avg       0.78      0.78      0.77      1800

Training BERT Model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.535200,0.454494
2,0.327600,0.459672
3,0.211100,0.735647


BERT Model Evaluation:
{'eval_loss': 0.45449379086494446, 'eval_runtime': 13.6589, 'eval_samples_per_second': 131.782, 'eval_steps_per_second': 16.473, 'epoch': 3.0}
  adding: bert_hate_speech_model/ (stored 0%)
  adding: bert_hate_speech_model/config.json (deflated 49%)
  adding: bert_hate_speech_model/vocab.txt (deflated 53%)
  adding: bert_hate_speech_model/tokenizer_config.json (deflated 75%)
  adding: bert_hate_speech_model/special_tokens_map.json (deflated 42%)
  adding: bert_hate_speech_model/model.safetensors (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Install required libraries
!pip install scikit-learn transformers datasets torch pandas numpy

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset
from google.colab import files

# Check for GPU
print("Checking for GPU...")
print(torch.cuda.is_available())  # Should print True if GPU is enabled
!nvidia-smi  # Displays GPU info if available

# Upload the dataset manually to Colab
uploaded = files.upload()  # Upload 'train_en.tsv'

# Load the dataset (TSV format with tab delimiter)
dataset_path = list(uploaded.keys())[0]
df = pd.read_csv(dataset_path, sep='\t', on_bad_lines='warn')
print(f"Loaded {len(df)} samples")

# Basic preprocessing function
def preprocess_text(text):
    text = str(text).lower()
    text = ''.join(c for c in text if c.isalnum() or c.isspace())
    return text

df['text'] = df['text'].apply(preprocess_text)

# Split into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# --- Baseline Model: TF-IDF + Logistic Regression ---
print("Training Baseline Model (TF-IDF + Logistic Regression)...")
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train = vectorizer.fit_transform(train_df['text'])
X_test = vectorizer.transform(test_df['text'])
y_train = train_df['HS']
y_test = test_df['HS']

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)
print("Baseline Model Evaluation:")
print(classification_report(y_test, y_pred))

# --- Transformer Model: BERT ---
print("Training BERT Model...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

train_dataset = Dataset.from_pandas(train_df[['text', 'HS']].rename(columns={'HS': 'label'}))
test_dataset = Dataset.from_pandas(test_df[['text', 'HS']].rename(columns={'HS': 'label'}))

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()
eval_results = trainer.evaluate()
print("BERT Model Evaluation (Loss Metrics):")
print(eval_results)

# Get predictions for classification report
predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.argmax(-1)  # Convert model outputs to 0 or 1
y_test = test_dataset['label']  # True labels
print("BERT Classification Report:")
print(classification_report(y_test, y_pred))

# Save the model and download it
model.save_pretrained("bert_hate_speech_model")
tokenizer.save_pretrained("bert_hate_speech_model")
!zip -r bert_hate_speech_model.zip bert_hate_speech_model
files.download("bert_hate_speech_model.zip")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Saving train_en.tsv to train_en (1).tsv
Loaded 9000 samples
Training Baseline Model (TF-IDF + Logistic Regression)...
Baseline Model Evaluation:
              precision    recall  f1-score   support

           0       0.78      0.87      0.82      1060
           1       0.78      0.65      0.71       740

    accuracy                           0.78      1800
   macro avg       0.78      0.76      0.76      1800
weighted avg       0.78      0.78      0.77      1800

Training BERT Model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.482900,0.449898
2,0.366800,0.514901
3,0.170800,0.820243


BERT Model Evaluation (Loss Metrics):
{'eval_loss': 0.44989779591560364, 'eval_runtime': 13.9271, 'eval_samples_per_second': 129.244, 'eval_steps_per_second': 16.155, 'epoch': 3.0}
BERT Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.72      0.80      1060
           1       0.69      0.90      0.78       740

    accuracy                           0.79      1800
   macro avg       0.80      0.81      0.79      1800
weighted avg       0.82      0.79      0.79      1800

  adding: bert_hate_speech_model/ (stored 0%)
  adding: bert_hate_speech_model/config.json (deflated 49%)
  adding: bert_hate_speech_model/vocab.txt (deflated 53%)
  adding: bert_hate_speech_model/tokenizer_config.json (deflated 75%)
  adding: bert_hate_speech_model/special_tokens_map.json (deflated 42%)
  adding: bert_hate_speech_model/model.safetensors (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install transformers torch pandas

import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from google.colab import files

# Upload dev_en.tsv
uploaded = files.upload()
dev_path = list(uploaded.keys())[0]
dev_df = pd.read_csv(dev_path, sep='\t', on_bad_lines='warn')

# Load the saved model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert_hate_speech_model")
model = BertForSequenceClassification.from_pretrained("bert_hate_speech_model")

# Tokenize dev data
def tokenize_function(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

inputs = [tokenize_function(text) for text in dev_df['text']]
predictions = [model(**input).logits.argmax(-1).item() for input in inputs]

# Compare to true labels
from sklearn.metrics import classification_report
print(classification_report(dev_df['HS'], predictions))

Saving dev_en.tsv to dev_en (1).tsv
              precision    recall  f1-score   support

           0       0.87      0.69      0.77       573
           1       0.68      0.86      0.76       427

    accuracy                           0.76      1000
   macro avg       0.77      0.78      0.76      1000
weighted avg       0.79      0.76      0.76      1000



In [ ]:
!pip install transformers torch pandas

from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
from google.colab import files

# Upload test_en.tsv
uploaded = files.upload()  # Upload test_en.tsv
test_path = list(uploaded.keys())[0]

# Load with no header and set 5 column names
test_df = pd.read_csv(test_path, sep='\t', on_bad_lines='warn', header=None,
                      names=['id', 'text', 'HS', 'TR', 'AG'])

# Debug: Confirm it loaded correctly
print("Column names in test_en.tsv:", test_df.columns.tolist())
print("First few rows of test_en.tsv:\n", test_df.head())

# Load the saved model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert_hate_speech_model")
model = BertForSequenceClassification.from_pretrained("bert_hate_speech_model")

# Tokenize function
def tokenize_function(text):
    return tokenizer(str(text), padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Use the 'text' column, ensuring it's treated as strings
inputs = [tokenize_function(text) for text in test_df['text']]
predictions = [model(**input).logits.argmax(-1).item() for input in inputs]

# Save predictions
test_df['predicted_HS'] = predictions
test_df.to_csv("test_en_predictions.csv", index=False)
files.download("test_en_predictions.csv")

Saving test_en.tsv to test_en.tsv
Column names in test_en.tsv: ['id', 'text', 'HS', 'TR', 'AG']
First few rows of test_en.tsv:
       id                                               text  HS  TR  AG
0  31568  Bro is a bitch, fucking cunt https://t.co/dQ8J...   0   0   0
1  30734  I'm rewatching Breaking Bad & I could see why ...   1   1   0
2  31227  @charliekirk11 @asia742 364 down only about 12...   0   0   0
3  31224  >Harasses women and calls them bitch and crabb...   0   0   0
4  31139  Bitch you ain't gotta call my phone, matter fa...   0   0   0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install transformers torch pandas sklearn

from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
from google.colab import files
from sklearn.metrics import classification_report

# Upload test_en.tsv
uploaded = files.upload()  # Upload test_en.tsv
test_path = list(uploaded.keys())[0]

# Load with no header and set 5 column names
test_df = pd.read_csv(test_path, sep='\t', on_bad_lines='warn', header=None,
                      names=['id', 'text', 'HS', 'TR', 'AG'])

# Debug: Confirm it loaded correctly
print("Column names in test_en.tsv:", test_df.columns.tolist())
print("First few rows of test_en.tsv:\n", test_df.head())

# Load the saved model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert_hate_speech_model")
model = BertForSequenceClassification.from_pretrained("bert_hate_speech_model")

# Tokenize function
def tokenize_function(text):
    return tokenizer(str(text), padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Predict
inputs = [tokenize_function(text) for text in test_df['text']]
predictions = [model(**input).logits.argmax(-1).item() for input in inputs]

# Add predictions to DataFrame
test_df['predicted_HS'] = predictions

# Classification report
print("Classification Report for test_en.tsv:")
print(classification_report(test_df['HS'], test_df['predicted_HS']))

# Save predictions
test_df.to_csv("test_en_predictions.csv", index=False)
files.download("test_en_predictions.csv")

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


Saving test_en.tsv to test_en.tsv
Column names in test_en.tsv: ['id', 'text', 'HS', 'TR', 'AG']
First few rows of test_en.tsv:
       id                                               text  HS  TR  AG
0  31568  Bro is a bitch, fucking cunt https://t.co/dQ8J...   0   0   0
1  30734  I'm rewatching Breaking Bad & I could see why ...   1   1   0
2  31227  @charliekirk11 @asia742 364 down only about 12...   0   0   0
3  31224  >Harasses women and calls them bitch and crabb...   0   0   0
4  31139  Bitch you ain't gotta call my phone, matter fa...   0   0   0
Classification Report for test_en.tsv:
              precision    recall  f1-score   support

           0       0.84      0.09      0.16      1719
           1       0.44      0.98      0.61      1252

    accuracy                           0.46      2971
   macro avg       0.64      0.53      0.38      2971
weighted avg       0.67      0.46      0.35      2971



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **2025/03/09**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model_path = '/content/drive/MyDrive/Hate Speech Detection/bert_hate_speech_model'
test_file_path = '/content/drive/MyDrive/bert_hate_speech_model/test_en.tsv'


## Loading the model:

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

model_path = '/content/drive/MyDrive/Hate Speech Detection/bert_hate_speech_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)


# Loading the dataset:

In [ ]:
import pandas as pd

test_file_path = '/content/drive/MyDrive/Hate Speech Detection/test_en.tsv'
test_data = pd.read_csv(test_file_path, sep='\t')


# 1. Preprocess the Dataset:
Text Cleaning: Remove unnecessary characters, stop words, special symbols, and perform lowercasing.
Tokenization: Tokenize the text into words or subwords.
Padding and Truncation: Ensure all sequences have the same length for the model input.


In [ ]:
print(test_data.columns)

Index(['31568', 'Bro is a bitch, fucking cunt https://t.co/dQ8JagmNCZ', '0',
       '0.1', '0.2'],
      dtype='object')


In [ ]:
# Rename the columns to something more meaningful
test_data.columns = ['id', 'text', 'HS', 'irrelevant_1', 'irrelevant_2']

# Now apply the preprocessing to the 'text' column
test_data['input_ids'] = test_data['text'].apply(lambda x: preprocess_text(x)['input_ids'])


# 2. Prepare the Dataset for Evaluation:
Split the dataset into features (input_ids) and labels (HS).
Convert them into torch tensors for model evaluation.

In [ ]:
inputs = torch.cat(test_data['input_ids'].tolist(), dim=0)
labels = torch.tensor(test_data['HS'].values)

# 3. Make Predictions:
Evaluate the model on the test data by passing the tokenized text through it.
Get the model’s predictions (logits) and convert them to actual predictions.

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

model_path = '/content/drive/MyDrive/Hate Speech Detection/bert_hate_speech_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

In [ ]:
# Extracting labels and text
labels = test_data['HS'].values
texts = test_data['text'].values
from transformers import BertTokenizer
import torch

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

# Example function to preprocess and tokenize the text
def preprocess_text(text, max_length=128):
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
    return encoding

# Apply preprocessing to your test data
test_data['input_ids'] = test_data['text'].apply(lambda x: preprocess_text(x)['input_ids'])
inputs = torch.cat(test_data['input_ids'].tolist(), dim=0)


In [ ]:
def preprocess_text(text, max_length=128):
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )
    # Return both input_ids and attention_mask
    return encoding

# Apply preprocessing to your test data
test_data['input_ids'] = test_data['text'].apply(lambda x: preprocess_text(x)['input_ids'])
test_data['attention_mask'] = test_data['text'].apply(lambda x: preprocess_text(x)['attention_mask'])

# Combine the input_ids and attention_mask for model input
inputs = torch.cat(test_data['input_ids'].tolist(), dim=0)
attention_masks = torch.cat(test_data['attention_mask'].tolist(), dim=0)


## 2023/05/12

# Load the Model:

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

model_path = "/content/drive/MyDrive/Hate Speech Detection/bert_hate_speech_model"

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

# Load the model
model = BertForSequenceClassification.from_pretrained(model_path)
print("Model and tokenizer loaded successfully!")

Model and tokenizer loaded successfully!


# Evaluate on test_en.tsv:

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the test data
test_df = pd.read_csv('/content/drive/MyDrive/Hate Speech Detection/test_en.tsv', sep='\t')
test_df.columns = ['id', 'text', 'HS', 'irrelevant_1', 'irrelevant_2']
test_dataset = Dataset.from_pandas(test_df[['text', 'HS']])

# Tokenize the test data
tokenized_test = test_dataset.map(
    lambda x: tokenizer(x['text'], padding='max_length', truncation=True, max_length=128),
    batched=True
)
tokenized_test.set_format('torch', columns=['input_ids', 'attention_mask', 'HS'])

# Evaluate
from transformers import Trainer
from sklearn.metrics import accuracy_score
import torch

trainer = Trainer(model=model)
predictions = trainer.predict(tokenized_test)

# Extract predicted labels
pred_labels = torch.argmax(torch.tensor(predictions.predictions), dim=-1).numpy()
true_labels = tokenized_test['HS']

# Calculate accuracy
accuracy = accuracy_score(true_labels, pred_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Map:   0%|          | 0/2970 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sergeantsaab9 (sergeantsaab9-cardiff-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Test Accuracy: 50.94%


# Detailed Metrics

In [ ]:
from sklearn. metrics import classification_report
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.89      0.17      0.29      1718
           1       0.46      0.97      0.62      1252

    accuracy                           0.51      2970
   macro avg       0.67      0.57      0.46      2970
weighted avg       0.71      0.51      0.43      2970



#Test on a Sample Input (Optional)
To double-check the model’s behavior, run a sample inference:

In [ ]:
sample_text = "This is a Hateful comment that should be flagged."
inputs = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
label = "Hate Speech" if predictions.item() == 1 else "Non-Hate Speech"
print(f"Prediction: {label}")

Prediction: Non-Hate Speech


# SAVED TILL HERE

our model is functional, but the performance (51% accuracy, skewed recall) suggests room for improvement, especially for Non-Hate Speech detection. Adjust the Decision Threshold
The current threshold of 0.5 (where logits > 0.5 are classified as Class 1) might not suit the imbalance. Let’s try optimizing it:

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve

#get probabilities instead of hard predicitons
trainer = Trainer(model=model)
predictions = trainer.predict(tokenized_test)
probs = torch.softmax(torch.tensor(predictions.predictions), dim=-1)[:, 1].numpy()

#compute precision-recall curve
precision, recall, thresholds = precision_recall_curve(tokenized_test['HS'], probs)

#find the threshold that balances precisoin recall
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold:.2f}")
print(f"Max F1-Score: {f1_scores[optimal_idx]:.2f}")

Optimal Threshold: 0.76
Max F1-Score: 0.63


In [ ]:
# Use the optimal threshold for predictions
pred_labels_adjusted = (probs >= optimal_threshold).astype(int)
accuracy_adjusted = accuracy_score(true_labels, pred_labels_adjusted)
print(f"Adjusted Accuracy: {accuracy_adjusted * 100:.2f}%")
print(classification_report(true_labels, pred_labels_adjusted, target_names=['Non-Hate Speech', 'Hate Speech']))

Adjusted Accuracy: 55.82%
                 precision    recall  f1-score   support

Non-Hate Speech       0.80      0.31      0.45      1718
    Hate Speech       0.49      0.89      0.63      1252

       accuracy                           0.56      2970
      macro avg       0.64      0.60      0.54      2970
   weighted avg       0.67      0.56      0.53      2970



In [6]:
# Install required packages
!pip install datasets -q
!pip install nlpaug -q
!pip install nltk -q
!pip install wandb -q

# Verify installations
print("Installation completed. Please run the next cell to proceed.")

Installation completed. Please run the next cell to proceed.


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve, f1_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.utils.class_weight import compute_class_weight
from nlpaug.augmenter.word import SynonymAug
import nltk
import wandb

# Verify installations
try:
    from datasets import Dataset
    print("datasets package is successfully installed!")
except ImportError:
    print("datasets package installation failed. Please rerun the installation cell with !pip install datasets.")
try:
    import nlpaug
    print("nlpaug package is successfully installed!")
except ImportError:
    print("nlpaug package installation failed. Please rerun the installation cell with !pip install nlpaug.")
try:
    import wandb
    print("wandb package is successfully installed!")
except ImportError:
    print("wandb package installation failed. Please rerun the installation cell with !pip install wandb.")

nltk.download('wordnet')

# Initialize W&B
wandb.init(project="hate_speech_detection", name="roberta_optimized")

# Verify GPU
print("CUDA Available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")
torch.backends.cudnn.benchmark = True

# Mount Google Drive to access dataset files
from google.colab import drive
drive.mount('/content/drive')

# Load Dataset
train_dataset_path = "/content/drive/MyDrive/Hate Speech Detection/train_en.tsv"
test_dataset_path = "/content/drive/MyDrive/Hate Speech Detection/test_en.tsv"
df = pd.read_csv(train_dataset_path, sep="\t")
df_test = pd.read_csv(test_dataset_path, sep="\t", header=None)
df_test.columns = ['id', 'text', 'HS', 'TR', 'AG']

# Verify dataset loading
print("Train Dataset Shape:", df.shape)
print("Test Dataset Shape:", df_test.shape)
print("Sample of Train Data:\n", df.head())
print("Sample of Test Data:\n", df_test.head())

# Enhanced Preprocessing
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'[^\w\s#@]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_text'] = df['text'].apply(clean_text)
df_test['clean_text'] = df_test['text'].apply(clean_text)

# Light Data Augmentation
aug = SynonymAug(aug_p=0.2)  # 20% synonym replacement
augmented_texts = []
augmented_labels = []

for idx, row in df.iterrows():
    augmented_texts.append(row['clean_text'])
    augmented_labels.append(row['HS'])
    if row['HS'] == 1 and np.random.random() < 0.5:  # Augment 50% of Hate Speech samples
        try:
            augmented_text = aug.augment(row['clean_text'])[0]
            augmented_texts.append(augmented_text)
            augmented_labels.append(row['HS'])
        except Exception as e:
            print(f"Augmentation error for row {idx}: {e}")
            augmented_texts.append(row['clean_text'])
            augmented_labels.append(row['HS'])

df_augmented = pd.DataFrame({'clean_text': augmented_texts, 'HS': augmented_labels})

# Data Exploration
print("Missing Values:\n", df_augmented.isnull().sum())
print("Label Distribution After Augmentation:\n", df_augmented['HS'].value_counts())

# Vectorization for Traditional ML
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(df_augmented['clean_text'])
y = df_augmented['HS']
X_test = vectorizer.transform(df_test['clean_text'])

# Train-Test Split for Traditional ML
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Traditional Model Training & Evaluation
models = {
    "Logistic Regression": LogisticRegression(C=1.0, max_iter=500),
    "SVM": SVC(C=1.0, kernel='linear', probability=True),
    "Random Forest": RandomForestClassifier(n_estimators=200, max_depth=10)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    print(f"{name} Validation Accuracy: {accuracy_score(y_val, y_pred):.4f}")
    print(classification_report(y_val, y_pred))
    test_pred = model.predict(X_test)
    df_test[f'{name}_Prediction'] = test_pred

df_test.to_csv("test_predictions.csv", index=False)

# Deep Learning Setup with RoBERTa
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Precompute Tokenization
train_data = Dataset.from_pandas(df_augmented[['clean_text', 'HS']])
test_data = Dataset.from_pandas(df_test[['clean_text', 'HS']])

def tokenize_function(examples):
    return tokenizer(examples['clean_text'], padding='max_length', truncation=True, max_length=64)

tokenized_train = train_data.map(tokenize_function, batched=True, batch_size=32)
tokenized_test = test_data.map(tokenize_function, batched=True, batch_size=32)

# Split tokenized_train
train_val_split = tokenized_train.train_test_split(test_size=0.2, seed=42)
tokenized_train = train_val_split['train']
tokenized_val = train_val_split['test']

# Debug: Verify tokenized_val
print("Tokenized Val Dataset Columns Before Formatting:", tokenized_val.column_names)
print("Tokenized Val Dataset Sample Before Formatting:", tokenized_val[0] if tokenized_val else "Empty dataset")
print("Tokenized Val Dataset Length:", len(tokenized_val))

# Rename and format (step-by-step to avoid NoneType issue)
# Step 1: Rename column
tokenized_train = tokenized_train.rename_column("HS", "labels")
tokenized_val = tokenized_val.rename_column("HS", "labels")
tokenized_test = tokenized_test.rename_column("HS", "labels")
print("After renaming, Tokenized Val Columns:", tokenized_val.column_names)

# Step 2: Remove columns
tokenized_train = tokenized_train.remove_columns(['clean_text'])
tokenized_val = tokenized_val.remove_columns(['clean_text'])
tokenized_test = tokenized_test.remove_columns(['clean_text'])
print("After removing columns, Tokenized Val Columns:", tokenized_val.column_names)

# Step 3: Set format (in-place operation)
tokenized_train.set_format('torch')
tokenized_val.set_format('torch')
tokenized_test.set_format('torch')
print("After setting format, Tokenized Val Columns:", tokenized_val.column_names)
print("Tokenized Val Dataset Sample After Formatting:", tokenized_val[0] if tokenized_val else "Empty dataset")

# Compute class weights
train_labels = df_augmented['HS'].values
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to('cuda' if torch.cuda.is_available() else 'cpu')

# Define compute_metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')
    return {"accuracy": accuracy, "f1": f1}

# TrainingArguments
training_args = TrainingArguments(
    output_dir='./results_roberta',
    eval_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_dir='./logs_roberta',
    logging_steps=10,
    save_strategy='epoch',
    load_best_model_at_end=True,
    gradient_accumulation_steps=1,
    fp16=True,
    report_to="wandb",
)

# Initialize Trainer with base Trainer class
print("Initializing Trainer with eval_dataset:", "tokenized_val" if 'tokenized_val' in locals() else "Not found")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)

# Train the model
print("Starting training...")
trainer.train()
print("Training completed.")
eval_results = trainer.evaluate()
print("RoBERTa Evaluation Results:", eval_results)
wandb.log({"validation_loss": eval_results['eval_loss'], "validation_accuracy": eval_results['eval_accuracy']})

# Threshold Optimization
print("Optimizing threshold...")
predictions = trainer.predict(tokenized_val)
probs = torch.softmax(torch.tensor(predictions.predictions), dim=-1)[:, 1].numpy()
true_labels = tokenized_val['labels'].numpy()
precision, recall, thresholds = precision_recall_curve(true_labels, probs)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold:.4f}")
print(f"Max F1-Score: {f1_scores[optimal_idx]:.4f}")

# Test Predictions
print("Making test predictions...")
test_predictions = trainer.predict(tokenized_test)
probs_test = torch.softmax(torch.tensor(test_predictions.predictions), dim=-1)[:, 1].numpy()
pred_labels_test = (probs_test >= optimal_threshold).astype(int)
accuracy_test = accuracy_score(tokenized_test['labels'].numpy(), pred_labels_test)
print(f"RoBERTa Optimized Accuracy on Test Set: {accuracy_test * 100:.2f}%")
print(classification_report(tokenized_test['labels'].numpy(), pred_labels_test, target_names=['Non-Hate Speech', 'Hate Speech']))
wandb.log({"test_accuracy": accuracy_test})

# Backup the Model
!zip -r "/content/best_model.zip" "/content/drive/MyDrive/Hate Speech Detection/bert_hate_speech_model"
from google.colab import files
files.download("/content/best_model.zip")

# Finish W&B run
wandb.finish()
print("Script execution completed.")

datasets package is successfully installed!
nlpaug package is successfully installed!
wandb package is successfully installed!
CUDA Available: False
Device: CPU


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train Dataset Shape: (9000, 5)
Test Dataset Shape: (2971, 5)
Sample of Train Data:
     id                                               text  HS  TR  AG
0  201  Hurray, saving us $$$ in so many ways @potus @...   1   0   0
1  202  Why would young fighting age men be the vast m...   1   0   0
2  203  @KamalaHarris Illegals Dump their Kids at the ...   1   0   0
3  204  NY Times: 'Nearly All White' States Pose 'an A...   0   0   0
4  205  Orban in Brussels: European leaders are ignori...   0   0   0
Sample of Test Data:
       id                                               text  HS  TR  AG
0  31568  Bro is a bitch, fucking cunt https://t.co/dQ8J...   0   0   0
1  30734  I'm rewatching Breaking Bad & I could see why ...   1   1   0
2  31227  @charliekirk11 @asia742 364 down only about 12...   0   0   0
3  31224  >Harasses women and calls them bitch and crabb.

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger t

Augmentation error for row 0: 
**********************************************************************
  Resource averaged_perceptron_tagger_eng not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load taggers/averaged_perceptron_tagger_eng/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Augmentation error for row 1: 
**********************************************************************
  Resource averaged_perceptron_tagger_eng not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('averaged_perceptron_tagge

...

Map:   0%|          | 0/10910 [00:00<?, ? examples/s]

Map:   0%|          | 0/2971 [00:00<?, ? examples/s]

Tokenized Val Dataset Columns Before Formatting: ['clean_text', 'HS', 'input_ids', 'attention_mask']
Tokenized Val Dataset Sample Before Formatting: {'clean_text': '@colly_flowerr no that would be girls #womensuck', 'HS': 0, 'input_ids': [0, 1039, 438, 10147, 1215, 22547, 338, 117, 14, 74, 28, 1972, 849, 605, 1075, 1290, 5858, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
Tokenized Val Dataset Length: 2182
After renaming, Tokenized Val Columns: ['clean_text', 'labels', 'input_ids', 'attention_mask']
After removing columns, Tokenized Val Columns: ['labels', 'input_ids', 'attention_mask']
After setting format, Tokenized Val Columns: ['labels', 'input_ids', 'attention_mask']
Tokenized V

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
